# RFDiffusion Breakdown
Forward add noising: add 3D Gaussian noise to Ca coordinates, use Brownian motion to add noise to manifold of rotation matrix representing N-CA-C orientation   
Backward denoising: using RosettaFold as denoising engine (take coorindates with noise as input into RosettaFold and output the 'right' structure )  



## Key Modules
Embedding module:
1. positionalEmbedding: relative postion encoding [B, L, L, d]
2. class MSA_emb: input MSA [B,N,L,d_init] Seq [B,L]          
output: msa_embedding :[B,N,L,d_msa] pair_embedding: [B,L,L,d_pair]
3. extra_emb: embedding for extra msa
4. TemplatePairStack: use PairStr2Pair module to embed template structure [L, L, d]
5. TemplateTorsionStack: use MLP+ AttionwithBias to embed torsion
6. Templ_emb: embed features including
    2d: 37 distogram bin (rbf) +6 orientation  
        mask (missing/unaligned) 1
    1d: tiled AA (20 +GAP); confidence: 1; contacting or not (1)

7. recycleing: take MSA Pair state as input and update them with a engine that do not update parameteres





Attention module

1. FeedForwardLayer
2. Attention (conventional)
3. AttentionwithBias: adding bias and gate on top of 2
4. SequenceWeight
5. MSARowAttentionwithBias
6. MSAColAttention
7. MSAGlobalAttention: share key value accoress all attention heads
8. BaisedAxialAttention

Track_module (similar to Evoformer)
#1. MSA -> MSA update (biased attention. bias from pair & structure)
#2. Pair -> Pair update (biased attention. bias from structure)
#3. MSA -> Pair update (extract coevolution signal)
#4. Str -> Str update (node from MSA, edge from Pair)

Class MSAPairStr2MSA:
        '''
        Inputs:
            - msa: MSA feature (B, N, L, d_msa)
            - pair: Pair feature (B, L, L, d_pair)
            - rbf_feat: Ca-Ca distance feature calculated from xyz coordinates (B, L, L, 36)
            - xyz: xyz coordinates (B, L, n_atom, 3)
            - state: updated node features after SE(3)-Transformer layer (B, L, d_state)
        Output:
            - msa: Updated MSA feature (B, N, L, d_msa)

Class PairStr2Pair:
        inputs: pair [B, L, L, d_pair] 
                rbf_feat [B, L, L, d]

        operation:   row/ colum/ff, using rbf_feat as bias
        outputs: updated pair 

Class MSA2Pair:

